<a href="https://colab.research.google.com/github/colzh/ms-drug-discovery/blob/main/Docking_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Adapted from https://pubs.acs.org/doi/10.1021/acs.jchemed.1c00022

**Prepare working environment**

In [1]:
# Install conda using the new conda-colab library
# This will restart the kernel, so must be run before anything else
# After kernel restarts ("Session crashed with unknown reason"), run all cells after this
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Installing py3Dmol using pip
!pip install py3Dmol
#Installing biopython using pip
!pip install biopython
#Installing pdb2pqr v3.0 using pip
!pip install pdb2pqr
#We will also install kora for using RDkit
!pip install kora
#Install MGLtools and OpenBabel from
#the bioconda repository
!conda install -c conda-forge -c bioconda mgltools openbabel zlib --yes

#Download and extract Autodock Vina from SCRIPPS
#Then, we set up an alias for vina to be treated as a native binary
!wget https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz
!tar xzvf autodock_vina_1_1_2_linux_x86.tgz

#Download Autodock script that splits PDB files with multiple conformations
!wget https://raw.githubusercontent.com/jaimergp/autodocktools-prepare-py3k/master/AutoDockTools/Utilities24/prepare_pdb_split_alt_confs.py
#Move script to /usr/local/bin/
!mv prepare_pdb_split_alt_confs.py /usr/local/bin/prepare_pdb_split_alt_confs.py
# Set permissions to allow execution
!chmod 755 /usr/local/bin/prepare_pdb_split_alt_confs.py

# # Partie rajouté par BA_Mohamed --> téléchargement de prepare_ligands.py
# !wget https://raw.githubusercontent.com/sahrendt0/Scripts/master/docking_scripts/prepare_ligand4.py
# #Move script to /usr/local/bin/
# !mv prepare_ligand4.py /usr/local/bin/prepare_ligand4.py
# # Set permissions to allow execution
# !chmod 755 /usr/local/bin/prepare_ligand4.py

!wget https://raw.githubusercontent.com/sahrendt0/Scripts/master/docking_scripts/prepare_receptor4.py
!mv prepare_receptor4.py /usr/local/bin/prepare_receptor4.py
!chmod 755 /usr/local/bin/prepare_receptor4.py


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
✨🍰✨ Everything looks OK!
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Error while loading conda entry point: conda-libma

In [ ]:
# Add extra path because colab can't find stuff
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

import json
import py3Dmol
import os
import zipfile
from Bio.PDB import *
from pathlib import Path
import subprocess
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')


def run_in_shell(command):
  stdout = subprocess.check_output(command, shell=True)
  print(stdout.decode('UTF-8'))

In [ ]:
# CAUTION: This script will likely fail if your PDB file has multiple instances of the desired ligand

pdb_id = ['7eo4']                          # Name of receptor-ligand complex in PDB (must be lowercase)
pdb_ligand_id = 'J8C'                      # Name of ligand in PDB

# One of the following must be provided:
drugbank_ligand_id = 'DB12371'             # Name of ligand in drugbank

ligand_smiles = ['CCC1=CC(=CC=C1CN1CC(C1)C(O)'] # SMILES string of ligand
# ligand_smiles = ['CCC1=CC(=CC=C1CN1CC(C1)C(O)=O)C(\C)=N\OCC1=CC=C(C2CCCCC2)C(=C1)C(F)(F)F'] # SMILES string of ligand
with open('drive/MyDrive/controle/ligands.txt', 'r') as f:
    lines = f.read()
    lines = lines[:lines.rfind(',')]
    ligand_smiles.extend(lines.split(',\n'))

print(f'Ligand SMILES: {ligand_smiles}')

###############################################################################
print('Preparing receptor file...')

#Downloading PDB file using biopython
pdbl = PDBList()
for s in pdb_id:
  pdbl.retrieve_pdb_file(s, pdir='.', file_format ="pdb", overwrite=True) #https://userguide.mdanalysis.org/1.0.0/formats/reference/pdb.html
  os.rename("pdb"+s+".ent", "orig.pdb")

# files.download(f"/content/orig.pdb")

# Extract protein from PDB file
with open(f"/content/clean.pdb","w") as g:
  f = open(f"orig.pdb",'r')
  for line in f:
    row = line.split()
    if row[0] == "ATOM":
      g.write(line)
    elif row[0] == "TER":
      g.write("TER\n")
  g.write("END")
  print("file successfully created")

# Extract ligand from PDB file
with open("ligand_solved.pdb","w") as g:
  f = open(f"orig.pdb",'r')
  for line in f:
    row = line.split()
    if pdb_ligand_id in row:
      g.write(line)
  g.write("END")

try:
  #Parameterizing and adding Gasteiger charges into our protein using MGLtools
  !echo "ok"
  !prepare_receptor4.py -r /content/clean.pdb -o /content/receptor.pdbqt -A hydrogens -U nphs_lps -v
except:
  # Split receptor file if contains multiple conformations (will use first one for docking)
  !prepare_pdb_split_alt_confs.py -r /content/clean.pdb -o /content/clean_singleconf
  #Parameterizing and adding Gasteiger charges into our protein using MGLtools
  !python prepare_receptor4.py -r clean_singleconf_A.pdb -o /content/receptor.pdbqt -A hydrogens -U nphs_lps -v
print(f'Receptor preparation complete. Saved to receptor.pdbqt')

###############################################################################

#  another commment here

print('\nPreparing ligand file...')

if ligand_smiles:
    for i, smile in enumerate(ligand_smiles):
      with open(f"ligand{i}.smiles", "w") as f:
        f.write(smile)
else:
  #Downloading ligand from the DrugBank database in SMILES format
  run_in_shell(f'wget https://www.drugbank.ca/structures/small_molecule_drugs/{drugbank_ligand_id}.smiles')
  ligand_smiles = Path(f"{drugbank_ligand_id}.smiles").read_text()




# ##################
# # OPTION 1: Converting ligand from SMILES into a 3D PDB format
# # run_in_shell(f'obabel {drugbank_ligand_id}.smiles -O ligand.mol2 --gen3d best -p 7.4 --canonical')
# ##################
# # OPTION 2: Converting ligand from SMILES into a 3D MOL2 format and perform an energy minimization of the conformer using the GAFF forcefield
# run_in_shell(f'obabel {drugbank_ligand_id}.smiles -O ligand.mol2 --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd')
# for i, smile in tqdm(enumerate(ligand_smiles)):
#   run_in_shell(f'obabel ligand{i}.smiles -O ligand{i}.mol2 --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd')
#   os.remove(f"ligand{i}.smiles")
# ##################
# # OPTION 3: Converting ligand from SMILES into a 3D MOL2 format and perform a weighted rotor search for lowest energy conformer
# # run_in_shell(f'obabel {drugbank_ligand_id}.smiles -O ligand.mol2 --gen3d --best --canonical --conformers --weighted --nconf 50 --ff GAFF')

# ################################################################################
#Parameterizing and adding Gasteiger charges into our ligand using MGLtools
# !prepare_ligand4.py -l ligand.mol2 -o ligand.pdbqt -U nphs_lps -v
# os.remove("ligand.mol2")


# # run_in_shell(f'obabel {drugbank_ligand_id}.smiles -O ligand.mol2 --gen3d best -p 7.4 --canonical')
# # run_in_shell(f'obabel {drugbank_ligand_id}.smiles -O ligand.mol2 --gen3d --best --canonical --conformers --weighted --nconf 50 --ff GAFF')
# run_in_shell(f'obabel {drugbank_ligand_id}.smiles -O ligand.mol2 --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd')



#here iw ill make some comments and i will deactivate it




# for i, smile in enumerate(ligand_smiles):
#   run_in_shell(f'prepare_ligand4.py -l ligand{i}.mol2 -o ligand{i}.pdbqt -U nphs_lps -v')
#   os.remove(f"ligand{i}.mol2")


# # Code to zip and download pdbqt files
zip_file = zipfile.ZipFile('/content/pdbqt_files.zip', 'w')

for filename in os.listdir():
  # Check if the file ends with .pdbqt
  if filename.endswith('.pdbqt'):
    # Write the file to the zip file
    zip_file.write(filename)

# Close the zip file
zip_file.close()

# Download the zip file
# files.download('/content/pdbqt_files.zip')



In [ ]:
# Move the blue box until it completely covers the green ligand

#First, we define the grid box
def definegrid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({'center':{'x':bxi,'y':byi,'z':bzi},'dimensions': {'w':bxf,'h':byf,'d':bzf},'color':'blue','opacity': 0.6})

#Next, we define how the protein will be shown in py3Dmol
#Note that we are also adding a style representation for active site residues
def viewprot(object,prot_PDBfile,resids):
  # Load protein
  object.addModel(open(prot_PDBfile, 'r').read(),'pdb')
  object.setStyle({'cartoon': {'color':'grey'}})
  # object.addStyle({'resi':resids},{'stick':{'colorscheme':'greenCarbon'}})
  # Loading the experimentally solved ligand
  object.addModel(open('ligand_solved.pdb', 'r').read(),'pdb')
  object.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})

#Lastly, we combine the box grid and protein into a single viewer
def viewprotgrid(prot_PDBfile,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10):
  mol_view = py3Dmol.view(1000,600)
  definegrid(mol_view,bxi,byi,bzi,bxf,byf,bzf)
  viewprot(mol_view,prot_PDBfile,resids)
  mol_view.setBackgroundColor('0xffffff')
  mol_view.zoomTo()
  mol_view.show()

from ipywidgets import interact,fixed,IntSlider
import ipywidgets
interact(viewprotgrid,
         prot_PDBfile = [f'orig.pdb'],
         resids = [],
         bxi=ipywidgets.IntSlider(value=128, min=-100,max=150, step=1),
         byi=ipywidgets.IntSlider(value=135, min=-100,max=150, step=1),
         bzi=ipywidgets.IntSlider(value=166, min=-100,max=200, step=1),
         bxf=ipywidgets.IntSlider(value=30, min=0,max=30, step=1),
         byf=ipywidgets.IntSlider(value=30, min=0,max=30, step=1),
         bzf=ipywidgets.IntSlider(value=30, min=0,max=30, step=1))

In [ ]:
# Make empty dictionary for binding affinities
ligand_dict = {}

for ligand in ligand_smiles:
    ligand_dict[ligand] = {}

# Extract all pdbqt_files
with zipfile.ZipFile('/content/pdbqt_files.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
bxi = 128
byi = 135
bzi = 166
bxf = 30
byf = 30
bzf = 30

for i, smile in enumerate(ligand_smiles):
# Create config file for docking
  with open(f"/content/config{i}.txt","w") as f:
    f.write("#CONFIGURATION FILE (options not used are commented) \n")
    f.write("\n")
    f.write("#INPUT OPTIONS \n")
    f.write(f"receptor = receptor.pdbqt \n")
    f.write(f"ligand = ligand{i}.pdbqt \n")
    f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
    f.write("#SEARCH SPACE CONFIGURATIONS \n")
    f.write("#Center of the box (values bxi, byi and bzi) \n")
  #CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
    f.write(f"center_x = {bxi} \n")
    f.write(f"center_y = {byi} \n")
    f.write(f"center_z = {bzi} \n")
  #CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
    f.write("#Size of the box (values bxf, byf and bzf) \n")
    f.write(f"size_x = {bxf} \n")
    f.write(f"size_y = {byf} \n")
    f.write(f"size_z = {bzf} \n")
    f.write("#OUTPUT OPTIONS \n")
    f.write("#out = \n")
    f.write("#log = \n")
    f.write("\n")
    f.write("#OTHER OPTIONS \n")
    f.write("#cpu =  \n")
    f.write("#exhaustiveness = \n")
    f.write("#num_modes = \n")
    f.write("#energy_range = \n")
    f.write("#seed = ")
############################################################################
#Executing AutoDock Vina with our configuration file

  run_in_shell(f'/content/autodock_vina_1_1_2_linux_x86/bin/vina --config config{i}.txt --out output{i}.pdbqt --log log{i}.txt')


  #Use babel to convert our Autodock Vina results from pdbqt into pdb
  run_in_shell(f'obabel -ipdbqt output{i}.pdbqt -opdb -O output{i}.pdb -m')

  with open(f'output{i}1.pdb', 'r') as f:
    for line in f:
      if line.startswith("REMARK VINA RESULT"):
        words = line.split()
        ligand_dict[list(ligand_dict.keys())[i]] = float(words[3])

  with open("ligand_dict.txt", 'w') as f:
    json.dump(ligand_dict, f)

  os.remove(f'config{i}.txt')
  os.remove(f'output{i}.pdbqt')

  for k in range(2, 10):
    try:
      os.remove(f'output{i}{k}.pdb')
    except:
      pass

files.download("/content/ligand_dict.txt")
# ##############################################################################
# # Visualize autodock docking pose (grey) vs. true docking pose (green)
# view=py3Dmol.view()
# view.setBackgroundColor('white')
# #Loading the target protein as PDB file
# view.addModel(open(f'clean.pdb', 'r').read(),'pdb')
# view.setStyle({'cartoon': {'color':'grey'}})
# view.zoomTo()
# #Loading the docking pose
# view.addModel(open('output01.pdb', 'r').read(),'pdb')
# view.setStyle({'model':1},{'stick':{'colorscheme':'greenCarbon'}})
# #Loading the experimentally solved binding mode
# # view.addModel(open('ligand_solved.pdb', 'r').read(),'pdb')
# # view.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
# # view.show()

In [ ]:
# Redock high-affinity pdbqt files

# Extract all files
with zipfile.ZipFile('/content/pdbqt_files.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

# with open('ligand_dict.txt', 'r') as f:
#     data = json.load(f)

# filtered_data = {key: value for key, value in data.items() if isinstance(value, (int, float))}

# # Get the binding energy of the first SMILES string
# first_smiles = list(filtered_data.keys())[0]
# first_binding_energy = filtered_data[first_smiles]

# # Initialize a list to store the indices of the SMILES strings with larger binding energies
# indices = []

# # Iterate over the dictionary and find the indices of the SMILES strings with larger binding energies
# for i, (smiles, energy) in enumerate(filtered_data.items()):
#     if energy < first_binding_energy:
#         indices.append(i)

indices = [346]

for file in os.listdir('/content'):
  if file.startswith('ligand') and file.endswith('.pdbqt'):
    index = int(file[6:-6])
    if index not in indices:
      os.remove(file)

bxi = 128
byi = 135
bzi = 166
bxf = 30
byf = 30
bzf = 30

for i in range(len(indices)):
# Create config file for docking
  with open(f"config{indices[i]}.txt","w") as f:
    f.write("#CONFIGURATION FILE (options not used are commented) \n")
    f.write("\n")
    f.write("#INPUT OPTIONS \n")
    f.write(f"receptor = receptor.pdbqt \n")
    f.write(f"ligand = ligand{indices[i]}.pdbqt \n")
    f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
    f.write("#SEARCH SPACE CONFIGURATIONS \n")
    f.write("#Center of the box (values bxi, byi and bzi) \n")
  #CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
    f.write(f"center_x = {bxi} \n")
    f.write(f"center_y = {byi} \n")
    f.write(f"center_z = {bzi} \n")
  #CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
    f.write("#Size of the box (values bxf, byf and bzf) \n")
    f.write(f"size_x = {bxf} \n")
    f.write(f"size_y = {byf} \n")
    f.write(f"size_z = {bzf} \n")
    f.write("#OUTPUT OPTIONS \n")
    f.write("#out = \n")
    f.write("#log = \n")
    f.write("\n")
    f.write("#OTHER OPTIONS \n")
    f.write("#cpu =  \n")
    f.write("#exhaustiveness = \n")
    f.write("#num_modes = \n")
    f.write("#energy_range = \n")
    f.write("#seed = ")
############################################################################
#Executing AutoDock Vina with our configuration file

for i in range(len(indices)):
  run_in_shell(f'/content/autodock_vina_1_1_2_linux_x86/bin/vina --config config{indices[i]}.txt --out output{indices[i]}.pdbqt --log log{indices[i]}.txt')

  #Use babel to convert our Autodock Vina results from pdbqt into pdb
  run_in_shell(f'obabel -ipdbqt output{indices[i]}.pdbqt -opdb -O output{indices[i]}.pdb -m')

  os.remove(f'config{indices[i]}.txt')
  os.remove(f'output{indices[i]}.pdbqt')

  for k in range(2, 10):
    try:
      os.remove(f'output{indices[i]}{k}.pdb')
    except:
      pass

for filename in os.listdir('/content'):
  if filename.startswith('output') and filename.endswith('1.pdb'):
    new_name = filename.replace("1.pdb", ".pdb")
    os.rename(filename, new_name)

In [ ]:
# Zip and download pdb files
zip_file = zipfile.ZipFile('pdb_files.zip', 'w')

for filename in os.listdir():
  # Check if the file ends with .pdb
  if filename.endswith('.pdb'):
    # Write the file to the zip file
    zip_file.write(filename)

zip_file.close()
files.download('pdb_files.zip')

In [ ]:
# Zip and download txt files
zip_file = zipfile.ZipFile('log_files.zip', 'w')

for filename in os.listdir():
  # Check if the file ends with .txt
  if filename.endswith('.txt'):
    # Write the file to the zip file
    zip_file.write(filename)

zip_file.close()
files.download('log_files.zip')

In [ ]:
view=py3Dmol.view()
view.setBackgroundColor('white')
#Loading the target protein as PDB file
view.addModel(open(f'clean.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'gray'}})
view.zoomTo()
# #Loading the docking pose
view.addModel(open('output271.pdb', 'r').read(),'pdb')
view.setStyle({'model':1},{'stick':{'colorscheme':'orangeCarbon'}})


#Loading the experimentally solved binding mode
# view.addModel(open('ligand_solved.pdb', 'r').read(),'pdb')
# view.setStyle({'model':1},{'stick':{'colorscheme':'greenCarbon'}})
view.show()

In [ ]:
# Clear directory
for file in os.listdir('/content'):
  if file.split('.')[-1] in ['smiles', 'pdbqt', 'mol2', 'pdb'] or file.startswith("config") or file.startswith("log"):
    os.remove(file)

**Redocking to confirm accuracy**

In [ ]:
# # Redocking

# # Prepare solved ligand for redocking
# !prepare_ligand4.py -l ligand_solved.pdb -o ligand_solved.pdbqt -A hydrogens -U nphs_lps -v

# # Generate new config file
# with open("config_redock","w") as f:
#   f.write("#CONFIGURATION FILE (options not used are commented) \n")
#   f.write("\n")
#   f.write("#INPUT OPTIONS \n")
#   f.write("receptor = receptor.pdbqt \n")
#   f.write("ligand = ligand_solved.pdbqt \n")
#   f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
#   f.write("#SEARCH SPACE CONFIGURATIONS \n")
#   f.write("#Center of the box (values bxi, byi and bzi) \n")
# #CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
#   f.write(f"center_x = {bxi} \n")
#   f.write(f"center_y = {byi} \n")
#   f.write(f"center_z = {bzi} \n")
# #CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
#   f.write("#Size of the box (values bxf, byf and bzf) \n")
#   f.write(f"size_x = {bxf} \n")
#   f.write(f"size_y = {byf} \n")
#   f.write(f"size_z = {bzf} \n")
#   f.write("#OUTPUT OPTIONS \n")
#   f.write("#out = \n")
#   f.write("#log = \n")
#   f.write("\n")
#   f.write("#OTHER OPTIONS \n")
#   f.write("#cpu =  \n")
#   f.write("#exhaustiveness = \n")
#   f.write("#num_modes = \n")
#   f.write("#energy_range = \n")
#   f.write("#seed = ")

# #Executing AutoDock Vina with our configuration file
# !/content/autodock_vina_1_1_2_linux_x86/bin/vina --config config_redock --out output_redock.pdbqt --log log_redock.txt

# #Use babel to convert our Autodock Vina results from pdbqt into pdb
# run_in_shell(f'obabel -ipdbqt output_redock.pdbqt -opdb -O output_redock.pdb -m')

# view=py3Dmol.view()
# view.setBackgroundColor('white')
# #Loading the target protein as PDB file
# view.addModel(open(f'clean.pdb', 'r').read(),'pdb')
# view.setStyle({'cartoon': {'color':'grey'}})
# view.zoomTo()
# #Loading the docking pose (grey)
# view.addModel(open('output_redock1.pdb', 'r').read(),'pdb')
# view.setStyle({'model':1},{'stick':{}})
# #Loading the experimentally solved binding mode (green)
# view.addModel(open('ligand_solved.pdb', 'r').read(),'pdb')
# view.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
# view.show()